In [40]:
import requests
import csv
import os
import urllib
import sys
import importlib
import pandas as pd
from shapely.geometry import Point, Polygon
import time
import datetime
from datetime import datetime
from datetime import timedelta  
from time import sleep
import geopandas 

sys.path.insert(1, '../utils')

import utils

In [41]:
s = requests.Session()

In [80]:
importlib.reload(utils)

<module 'utils' from '../utils/utils.py'>

In [81]:
region = 'District of Columbia'
extent = 'region'
county_ids = utils.county_ids.get_county_ids(region, extent)
input_date = '2020-06-10'
input_ts = time.mktime(datetime.strptime(input_date, "%Y-%m-%d").timetuple())
xmin, xmax, ymin, ymax = utils.geometry.osm_bounds(region, county_ids, extent, file = False, raw = True)

url = 'https://api.transitfeeds.com'

In [82]:
# bounding box

coords = [(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin)]
poly = Polygon(coords)

In [83]:
# api key

#key = '9efe3e4d-9b04-467c-9c72-0cc3cf666fd0'
key = 'f2a91a7e-154d-434a-8083-2cd18e25f3d2'

In [84]:
output_feed_info = [list(['operator_name', 'operator_website', 'operator_location', 'operator_metro', 'feed_id', 'date_query', 
                         'date_fetched', 'earliest_calendar_date', 'latest_calendar_date', 'transit_feeds_url'])]

In [85]:
# gets all available locations on transit feeds

response = s.get(
    url+'/v1/getLocations',
    params = {'key': key
       
    }
)
locations = response.json()['results']['locations']
sleep(15)

In [86]:
# filters locations to only those within bounding box

location_ids = []
for operator in locations:
    pt = Point(operator['lng'], operator['lat'])
    if pt.within(poly) is True: 
        location_ids.append(operator['id'])

In [87]:
os.makedirs("../gtfs/feeds_" + input_date, exist_ok=True)

In [88]:
# loops through all locations
error_count = 0
id_lst = []
for ids in location_ids:
    for attempt in range(4):
        try: 
            response = s.get(
                url+'/v1/getFeeds',
                params = {'key': key, 'location' : ids, 'type': 'gtfs', 'limit': 200

                }
            )
            feeds = response.json()['results']['feeds']
            for agencies in feeds:
                ts = agencies['latest']['ts']
                dt_str = str(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d'))
                name = agencies['t']
                loc = agencies['l']['t']
                dt_fetched = str(datetime.utcfromtimestamp(ts).strftime('%Y%m%d'))
                id_lst.append(list([agencies['id'], dt_fetched, name, loc, dt_str]))
            sleep(1)
            break
        except:
            print(attempt)
            sleep(10)


In [89]:
gtfs_url = 'https://transitfeeds.com/p/'

In [90]:
for agency in id_lst:
    print(agency[0])
    
    # to account for timezone issues
    for attempt in range(3):
        try:
            if attempt == 0:
                url = gtfs_url + agency[0] + '/' + agency[1] + '/download'
                urllib.request.urlretrieve(url, "../gtfs/feeds_" + input_date + 
                                       "/" + agency[2] + '-' + input_date + ".zip")
                break
            elif attempt == 1:
                bkwd_dt = str((datetime.strptime(id_lst[0][1], '%Y%m%d') + timedelta(days=1)).date().strftime('%Y%m%d'))   
                url = gtfs_url + agency[0] + '/' + bkwd_dt + '/download'
                urllib.request.urlretrieve(url, "../gtfs/feeds_" + input_date + 
                                       "/" + agency[2] + '-' + input_date + ".zip")
                break
            elif attempt == 2:
                fwd_dt = str((datetime.strptime(id_lst[0][1], '%Y%m%d') - timedelta(days=1)).date().strftime('%Y%m%d'))   
                url = gtfs_url + agency[0] + '/' + fwd_dt + '/download'
                urllib.request.urlretrieve(url, "../gtfs/feeds_" + input_date + 
                                       "/" + agency[2] + '-' + input_date + ".zip")
                break
            else:
                print('Skipping due to error')
        except:
            pass

alexandria-transit-company/423
arlington-transit/149
carroll-area-transit-system/569
fairfax-cue/1042
fairfax-connector/295
central-maryland-regional-transit/297
saint-marys-transit-system/1147
frederick-county-transit/1082
washington-county-transit/1081
delmarva-community-transit/1079
charles-county-vango/1078
calvert-county-public-transportation/1077
charm-city-circulator/1041
regional-transportation-agency-of-central-maryland/574
mta-maryland/247
montgomery-county-department-of-transportation/235
prince-georges-county-thebus/1052
wmata/884
wmata/883
wmata/75
prtc/357


In [91]:
with open("../gtfs/feeds_" + input_date + "/" + region + "_feed_info_" + input_date + ".csv", "w") as csvfile:
    writer = csv.writer(csvfile)
    for row in id_lst:
        writer.writerow(row)